In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import joblib

# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data = pd.read_csv('0727_masking_labeling_data.csv')

selected_columns1 = ['title_morphed','text_morphed', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성 민원','출처']
data1 = data[selected_columns1][data.aggr == 0]
data2 = data[selected_columns1][data.aggr == 1]
data1 = data1[300:812]
data2 = data2

data2['출처'].value_counts()

출처
부산광역시_금정구      118
서울특별시_영등포구      73
경상북도_영덕군        49
강원특별자치도_강릉시     34
충청남도_청양군        30
강원특별자치도_고성군     30
충청남도_부여군        26
경상북도_청송군        26
광주광역시_동구        21
제주도_제주시         21
대구광역시_달서구       19
강원특별자치도_태백시     19
경상북도_고령군        15
제주도_서귀포시        14
경상북도_영양군         9
경상북도_울릉군         8
Name: count, dtype: int64

## TfidfVectorizer 마스킹 X, 파라미터 default - best

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import joblib

# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data = pd.read_csv('0727_masking_labeling_data.csv')

selected_columns1 = ['title_morphed','text_morphed', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성 민원','출처']
data1 = data[selected_columns1][data.aggr == 0]
data2 = data[selected_columns1][data.aggr == 1]
data1 = data1[300:812]
data2 = data2


data3 = pd.concat([data1, data2])
# 텍스트와 라벨 분리
X_train = data3[data3.출처 != '부산광역시_금정구']['title_morphed'] + ' ' + data3[data3.출처 != '부산광역시_금정구']['text_morphed']
X_test = data3[data3.출처 == '부산광역시_금정구']['title_morphed'] + ' ' + data3[data3.출처 == '부산광역시_금정구']['text_morphed']
# 원래의 'text_morphed_'와 'title' 컬럼 삭제
y_train = data3[data3.출처 != '부산광역시_금정구']['aggr']
y_test = data3[data3.출처 == '부산광역시_금정구']['aggr']
# 데이터 분할

# 파이프라인 정의
pipelines = {

    'extra_trees': Pipeline([
        ('vect', TfidfVectorizer()),
        ('et', ExtraTreesClassifier())
    ]),

    'ridge_classifier': Pipeline([
        ('vect', TfidfVectorizer()),
        ('ridge', RidgeClassifier())
    ]),

    'logistic_regression': Pipeline([
        ('vect', TfidfVectorizer()),
        ('lr', LogisticRegression())
    ]),

    'naive_bayes': Pipeline([
        ('vect', TfidfVectorizer()),
        ('nb', MultinomialNB())
    ]),

    'svm': Pipeline([
        ('vect', TfidfVectorizer()),
        ('svm', SVC())
    ]),


    'hard_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('hard_model',VotingClassifier([('LR',LogisticRegression()),('ridge',RidgeClassifier()),('svm',SVC()),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='hard'))

    ]),
    'soft_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('soft_model',VotingClassifier([('LR',LogisticRegression()),('svm',SVC(probability=True)),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='soft'))

    ])
    

}

# 모델 훈련 및 평가
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")

    # 텍스트 데이터를 TF-IDF 벡터로 변환
    X_train_tfidf = pipeline.named_steps['vect'].fit_transform(X_train)
    X_test_tfidf = pipeline.named_steps['vect'].transform(X_test)
    
    # 차원 출력
    print(f"{model_name} - X_train shape: {X_train_tfidf.shape}")
    print(f"{model_name} - X_test shape: {X_test_tfidf.shape}")
    
    # 모델 훈련
    pipeline.fit(X_train, y_train)
    
    # 교차검증 점수 계산
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    
    # 성능 출력
    train_score = pipeline.score(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    print(f"{model_name} Train set score: {train_score:.4f}")
    print(f"{model_name} Test set score: {test_score:.4f}")
    print(f"{model_name} Cross-validation scores: {cv_scores}")
    print(f"{model_name} Cross-validation mean score: {cv_scores.mean():.4f}\n")

    # 모델 저장
    # model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/', f'0727_aggr_textmorphed_{model_name}_tfidf_nonemasking_model.pkl')
    # joblib.dump(pipeline, model_save_path)

print("All models have been trained and saved.")

Training extra_trees...
extra_trees - X_train shape: (906, 12176)
extra_trees - X_test shape: (118, 12176)
extra_trees Train set score: 0.9989
extra_trees Test set score: 0.7034
extra_trees Cross-validation scores: [0.76373626 0.86187845 0.86740331 0.82872928 0.71270718]
extra_trees Cross-validation mean score: 0.8069

Training ridge_classifier...
ridge_classifier - X_train shape: (906, 12176)
ridge_classifier - X_test shape: (118, 12176)
ridge_classifier Train set score: 0.9967
ridge_classifier Test set score: 0.7373
ridge_classifier Cross-validation scores: [0.70879121 0.80662983 0.90055249 0.86187845 0.79005525]
ridge_classifier Cross-validation mean score: 0.8136

Training logistic_regression...
logistic_regression - X_train shape: (906, 12176)
logistic_regression - X_test shape: (118, 12176)
logistic_regression Train set score: 0.9525
logistic_regression Test set score: 0.6441
logistic_regression Cross-validation scores: [0.7032967  0.84530387 0.8839779  0.87292818 0.78453039]
log

# 마스킹 테스트


In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import joblib

# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data = pd.read_csv('0727_masking_labeling_data.csv')

selected_columns1 = ['title_morphed_masked','text_morphed_masked', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성 민원','출처']
data1 = data[selected_columns1][data.aggr == 0]
data2 = data[selected_columns1][data.aggr == 1]
data1 = data1[300:812]
data2 = data2


data3 = pd.concat([data1, data2])
# 텍스트와 라벨 분리
X_train = data3[data3.출처 != '부산광역시_금정구']['title_morphed_masked'] + ' ' + data3[data3.출처 != '부산광역시_금정구']['text_morphed_masked']
X_test = data3[data3.출처 == '부산광역시_금정구']['title_morphed_masked'] + ' ' + data3[data3.출처 == '부산광역시_금정구']['text_morphed_masked']
# 원래의 'text_morphed_'와 'title' 컬럼 삭제
y_train = data3[data3.출처 != '부산광역시_금정구']['aggr']
y_test = data3[data3.출처 == '부산광역시_금정구']['aggr']
# 데이터 분할

# 파이프라인 정의
pipelines = {

    'extra_trees': Pipeline([
        ('vect', TfidfVectorizer()),
        ('et', ExtraTreesClassifier())
    ]),

    'ridge_classifier': Pipeline([
        ('vect', TfidfVectorizer()),
        ('ridge', RidgeClassifier())
    ]),

    'logistic_regression': Pipeline([
        ('vect', TfidfVectorizer()),
        ('lr', LogisticRegression())
    ]),

    'naive_bayes': Pipeline([
        ('vect', TfidfVectorizer()),
        ('nb', MultinomialNB())
    ]),

    'svm': Pipeline([
        ('vect', TfidfVectorizer()),
        ('svm', SVC())
    ]),


    'hard_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('hard_model',VotingClassifier([('LR',LogisticRegression()),('ridge',RidgeClassifier()),('svm',SVC()),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='hard'))

    ]),
    'soft_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('soft_model',VotingClassifier([('LR',LogisticRegression()),('svm',SVC(probability=True)),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='soft'))

    ])
    

}

# 모델 훈련 및 평가
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")

    # 텍스트 데이터를 TF-IDF 벡터로 변환
    X_train_tfidf = pipeline.named_steps['vect'].fit_transform(X_train)
    X_test_tfidf = pipeline.named_steps['vect'].transform(X_test)
    
    # 차원 출력
    print(f"{model_name} - X_train shape: {X_train_tfidf.shape}")
    print(f"{model_name} - X_test shape: {X_test_tfidf.shape}")
    
    # 모델 훈련
    pipeline.fit(X_train, y_train)
    
    # 교차검증 점수 계산
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    
    # 성능 출력
    train_score = pipeline.score(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    print(f"{model_name} Train set score: {train_score:.4f}")
    print(f"{model_name} Test set score: {test_score:.4f}")
    print(f"{model_name} Cross-validation scores: {cv_scores}")
    print(f"{model_name} Cross-validation mean score: {cv_scores.mean():.4f}\n")

    # 모델 저장
    # model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/', f'0727_aggr_textmorphed_{model_name}_tfidf_nonemasking_model.pkl')
    # joblib.dump(pipeline, model_save_path)

print("All models have been trained and saved.")

Training extra_trees...
extra_trees - X_train shape: (906, 11616)
extra_trees - X_test shape: (118, 11616)
extra_trees Train set score: 0.9989
extra_trees Test set score: 0.7458
extra_trees Cross-validation scores: [0.75824176 0.86187845 0.82872928 0.84530387 0.74033149]
extra_trees Cross-validation mean score: 0.8069

Training ridge_classifier...
ridge_classifier - X_train shape: (906, 11616)
ridge_classifier - X_test shape: (118, 11616)
ridge_classifier Train set score: 0.9967
ridge_classifier Test set score: 0.7373
ridge_classifier Cross-validation scores: [0.70879121 0.80662983 0.90607735 0.83977901 0.79005525]
ridge_classifier Cross-validation mean score: 0.8103

Training logistic_regression...
logistic_regression - X_train shape: (906, 11616)
logistic_regression - X_test shape: (118, 11616)
logistic_regression Train set score: 0.9481
logistic_regression Test set score: 0.6610
logistic_regression Cross-validation scores: [0.7032967  0.83977901 0.87292818 0.83425414 0.7679558 ]
log

## TfidfVectorizer 명사제거, 마스킹 X, 파라미터 default - 0726

In [38]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.pipeline import Pipeline
from sklearn.ensemble import VotingClassifier
import joblib

# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data1 = pd.read_csv('0726_masking_labeling_data_0.csv')
data2 = pd.read_csv('0726_masking_labeling_data_1.csv')

data1 = data1.rename(columns={'text_remove_noun_morphed':'text_remove_noun_morphed'})
data2 = data2.rename(columns={'text_remove_noun_morphed':'text_remove_noun_morphed'})

data1 = data1.rename(columns={'저격성 민원':'저격성민원_'})
data2 = data2.rename(columns={'저격성 민원':'저격성민원_'})

selected_columns1 = ['text_remove_noun_morphed', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성민원_']
data1 = data1[selected_columns1]
data2 = data2[selected_columns1]
data1 = data1[:366]
data2 = data2[data2.aggr == 1]

data3 = pd.concat([data1, data2])
# 텍스트와 라벨 분리
data_contents = data3['text_remove_noun_morphed']
data_labeling = data3['aggr']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_contents, data_labeling, stratify=data_labeling, test_size=0.2, random_state=42)

# 파이프라인 정의
pipelines = {

    'extra_trees': Pipeline([
        ('vect', TfidfVectorizer()),
        ('et', ExtraTreesClassifier())
    ]),

    'ridge_classifier': Pipeline([
        ('vect', TfidfVectorizer()),
        ('ridge', RidgeClassifier())
    ]),

    'logistic_regression': Pipeline([
        ('vect', TfidfVectorizer()),
        ('lr', LogisticRegression())
    ]),

    'naive_bayes': Pipeline([
        ('vect', TfidfVectorizer()),
        ('nb', MultinomialNB())
    ]),

    'svm': Pipeline([
        ('vect', TfidfVectorizer()),
        ('svm', SVC())
    ]),


    'hard_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('hard_model',VotingClassifier([('LR',LogisticRegression()),('ridge',RidgeClassifier()),('svm',SVC()),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='hard'))

    ]),

    'soft_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('soft_model',VotingClassifier([('LR',LogisticRegression()),('svm',SVC(probability=True)),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='soft'))

    ])
    

}

# 모델 훈련 및 평가
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")

    # 텍스트 데이터를 TF-IDF 벡터로 변환
    X_train_tfidf = pipeline.named_steps['vect'].fit_transform(X_train)
    X_test_tfidf = pipeline.named_steps['vect'].transform(X_test)
    
    # 차원 출력
    print(f"{model_name} - X_train shape: {X_train_tfidf.shape}")
    print(f"{model_name} - X_test shape: {X_test_tfidf.shape}")
    
    # 모델 훈련
    pipeline.fit(X_train, y_train)
    
    # 교차검증 점수 계산
    cv_scores = cross_val_score(pipeline, X_train, y_train, cv=5, scoring='accuracy')
    
    # 성능 출력
    train_score = pipeline.score(X_train, y_train)
    test_score = pipeline.score(X_test, y_test)
    print(f"{model_name} Train set score: {train_score:.4f}")
    print(f"{model_name} Test set score: {test_score:.4f}")
    print(f"{model_name} Cross-validation scores: {cv_scores}")
    print(f"{model_name} Cross-validation mean score: {cv_scores.mean():.4f}\n")
    
    # 모델 저장
    model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/', f'aggr_{model_name}_tfidf_nonemasking_model.pkl')
    joblib.dump(pipeline, model_save_path)

print("All models have been trained and saved.")

Training extra_trees...
extra_trees - X_train shape: (585, 2620)
extra_trees - X_test shape: (147, 2620)
extra_trees Train set score: 0.9983
extra_trees Test set score: 0.7483
extra_trees Cross-validation scores: [0.75213675 0.74358974 0.73504274 0.7008547  0.73504274]
extra_trees Cross-validation mean score: 0.7333

Training ridge_classifier...
ridge_classifier - X_train shape: (585, 2620)
ridge_classifier - X_test shape: (147, 2620)
ridge_classifier Train set score: 0.9846
ridge_classifier Test set score: 0.8027
ridge_classifier Cross-validation scores: [0.76923077 0.73504274 0.70940171 0.67521368 0.70940171]
ridge_classifier Cross-validation mean score: 0.7197

Training logistic_regression...
logistic_regression - X_train shape: (585, 2620)
logistic_regression - X_test shape: (147, 2620)
logistic_regression Train set score: 0.9248
logistic_regression Test set score: 0.7823
logistic_regression Cross-validation scores: [0.74358974 0.70940171 0.73504274 0.73504274 0.70940171]
logistic_

## TfidfVectorizer 마스킹 X, 파라미터 default - best 그리드 서치

In [2]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC
from sklearn.ensemble import ExtraTreesClassifier, VotingClassifier
from sklearn.pipeline import Pipeline
import joblib

# 데이터 불러오기
os.chdir('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/라벨링 데이터')

data = pd.read_csv('0727_masking_labeling_data.csv')

selected_columns1 = ['text_morphed','title_morphed', 'aggr', '욕설_모욕', '비꼼_시비', '성희롱', '요지불명', '저격성 민원']
data1 = data[selected_columns1][data.aggr == 0]
data2 = data[selected_columns1][data.aggr == 1]
data1 = data1[300:812]
data2 = data2


data3 = pd.concat([data1, data2])
# 텍스트와 라벨 분리
data_contents = data3['text_morphed'] + ' ' + data3['title_morphed']
# 
# 원래의 'text_morphed_'와 'title' 컬럼 삭제
data_labeling = data3['aggr']

# 데이터 분할
X_train, X_test, y_train, y_test = train_test_split(data_contents, data_labeling, stratify=data_labeling, test_size=0.2, random_state=42)

# 파이프라인 정의
pipelines = {
    'extra_trees': Pipeline([
        ('vect', TfidfVectorizer()),
        ('et', ExtraTreesClassifier())
    ]),
    'ridge_classifier': Pipeline([
        ('vect', TfidfVectorizer()),
        ('ridge', RidgeClassifier())
    ]),
    'logistic_regression': Pipeline([
        ('vect', TfidfVectorizer()),
        ('lr', LogisticRegression())
    ]),
    'naive_bayes': Pipeline([
        ('vect', TfidfVectorizer()),
        ('nb', MultinomialNB())
    ]),
    'svm': Pipeline([
        ('vect', TfidfVectorizer()),
        ('svm', SVC())
    ]),
    'hard_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('hard_model',VotingClassifier([('LR',LogisticRegression()),('ridge',RidgeClassifier()),('svm',SVC()),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='hard'))

    ]),

    'soft_model' : Pipeline([
        ('vect', TfidfVectorizer()),
        ('soft_model',VotingClassifier([('LR',LogisticRegression()),('svm',SVC(probability=True)),('et',ExtraTreesClassifier()),('nb',MultinomialNB())],voting='soft'))
    ])
}

# 하이퍼파라미터 그리드 설정
param_grids = {
    'extra_trees': {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'vect__max_df': [0.9, 1.0],
        'vect__min_df': [1, 2],
        'vect__max_features': [None, 1000],
        'et__n_estimators': [100],
        'et__max_features': ['auto', 'sqrt'],
        'et__max_depth': [None, 50],
    },
    'ridge_classifier': {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'vect__max_df': [0.9, 1.0],
        'vect__min_df': [1, 2],
        'vect__max_features': [None, 1000],
        'ridge__alpha': [0.1, 1, 10],
    },
    'logistic_regression': {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'vect__max_df': [0.9, 1.0],
        'vect__min_df': [1, 2],
        'vect__max_features': [None, 1000],
        'lr__C': [0.1, 1, 10],
        'lr__penalty': ['l1', 'l2'],
        'lr__solver': ['liblinear'],
    },
    'naive_bayes': {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'vect__max_df': [0.9, 1.0],
        'vect__min_df': [1, 2],
        'vect__max_features': [None, 1000],
        'nb__alpha': [0.1, 1, 10],
    },
    'svm': {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'vect__max_df': [0.9, 1.0],
        'vect__min_df': [1, 2],
        'vect__max_features': [None, 1000],
        'svm__C': [0.1, 1, 10],
        'svm__kernel': ['linear', 'rbf'],
        'svm__gamma': ['scale'],
    },
    'hard_model': {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'vect__max_df': [0.9, 1.0],
        'vect__min_df': [1, 2],
        'vect__max_features': [None, 1000],
        'hard_model__LR__C': [0.1, 1],
        'hard_model__ridge__alpha': [0.1, 1],
        'hard_model__svm__C': [0.1, 1],
    },
    'soft_model': {
        'vect__ngram_range': [(1, 1), (1, 2)],
        'vect__max_df': [0.9, 1.0],
        'vect__min_df': [1, 2],
        'vect__max_features': [None, 1000],
        'soft_model__LR__C': [0.1, 1],
        'soft_model__svm__C': [0.1, 1],
        'soft_model__et__n_estimators': [100],
        'soft_model__nb__alpha': [0.1, 1],
    },
}

# 모델 훈련 및 평가
for model_name, pipeline in pipelines.items():
    print(f"Training {model_name}...")

    # 그리드 서치 설정
    param_grid = param_grids[model_name]
    grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
    
    # 그리드 서치 훈련
    grid_search.fit(X_train, y_train)
    
    # 최적 하이퍼파라미터 출력
    print(f"Best parameters for {model_name}: {grid_search.best_params_}")
    
    # 최적 모델 평가
    best_model = grid_search.best_estimator_
    train_score = best_model.score(X_train, y_train)
    test_score = best_model.score(X_test, y_test)
    
    # 교차 검증 점수 계산
    cv_scores = cross_val_score(best_model, X_train, y_train, cv=5, scoring='accuracy')
    
    # 성능 출력
    print(f"{model_name} Train set score: {train_score:.4f}")
    print(f"{model_name} Test set score: {test_score:.4f}")
    print(f"{model_name} Cross-validation scores: {cv_scores}")
    print(f"{model_name} Cross-validation mean score: {cv_scores.mean():.4f}\n")
    
    # 모델 저장
    model_save_path = os.path.join('C:/Users/USER/Desktop/핵심역량 프로젝트/데이터/모델/', f'aggr_{model_name}_tfidf_nonemasking_model.pkl')
    joblib.dump(best_model, model_save_path)

print("All models have been trained and saved.")



Training extra_trees...


KeyboardInterrupt: 